In [ ]:
# 0) Install dependencies (run once per environment)
%pip install -U pip
%pip install -U langchain langchain-community langchain-core  langchain-groq sentence-transformers chromadb pypdf python-dotenv
%pip install -U ipywidgets
%pip install -U jupyter
%pip install -U ipywidgets tqdm
%pip install -U widgetsnbextension






Note: you may need to restart the kernel to use updated packages.


In [1]:
from dotenv import load_dotenv; load_dotenv()
import os; print("Has GROQ key?", bool(os.getenv("GROQ_API_KEY")))


Has GROQ key? True


In [60]:
!pip install pandas
import os
import tempfile
import uuid
import pandas as pd
from dotenv import load_dotenv 
import re


   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ------- -------------------------------- 2.1/11.3 MB 11.8 MB/s eta 0:00:01
   ----------------- ---------------------- 5.0/11.3 MB 12.6 MB/s eta 0:00:01
   -------------------------- ------------- 7.6/11.3 MB 12.4 MB/s eta 0:00:01
   ------------------------------------ --- 10.2/11.3 MB 12.5 MB/s eta 0:00:01
   ---------------------------------------- 11.3/11.3 MB 11.3 MB/s  0:00:01

   ---------------------------------------- 0/3 [pytz]
   ---------------------------------------- 0/3 [pytz]
   ---------------------------------------- 0/3 [pytz]
   ------------- -------------------------- 1/3 [tzdata]
   ------------- -------------------------- 1/3 [tzdata]
   ------------- -------------------------- 1/3 [tzdata]
   -------------------------- ------------- 2/3 [pandas]
   -------------------------- ------------- 2/3 [pandas]
   -------------------------- ------------- 2/3 [pandas]
   -------------------------- ---

In [2]:
# 1) Imports & config
import os
from dotenv import load_dotenv

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

# Load .env (expects GROQ_API_KEY)
load_dotenv()  # or load_dotenv(dotenv_path="C:/path/to/.env")
assert os.getenv("GROQ_API_KEY"), "GROQ_API_KEY not found. Add it to your .env."


In [4]:
%pip install -U "ipywidgets>=8" "tqdm>=4.66" "ipykernel>=6.25" "traitlets>=5.10"
# Only needed for classic Jupyter Notebook in a browser, NOT VS Code:
# %pip install -U widgetsnbextension==4.*


   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   -------------- ------------------------- 0.8/2.2 MB 4.8 MB/s eta 0:00:01
   ---------------------------- ----------- 1.6/2.2 MB 4.4 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 4.3 MB/s  0:00:00

   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   ---------------------------------------- 3/3 [ipywidgets]

Note: you may need to restart the kernel to use updated packages.


In [8]:
from tqdm.auto import tqdm

for _ in tqdm(range(100)):
    pass


c:\Users\Lenovo\Desktop\RAG-System\rag_llms\myenv311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



100%|██████████| 100/100 [00:00<?, ?it/s]


## Define our LLM Model


In [9]:
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")

In [10]:
from dotenv import load_dotenv
import os
from langchain_groq import ChatGroq

load_dotenv()  # loads GROQ_API_KEY from your .env
key = os.getenv("GROQ_API_KEY")
assert key and key.startswith("gsk_"), "GROQ_API_KEY missing or malformed."

# Either pass it explicitly...



In [11]:
# LLM (Groq) ---------------------------------------------------------------
from dotenv import load_dotenv; load_dotenv()
import os
from groq import Groq
from langchain_groq import ChatGroq

assert os.getenv("GROQ_API_KEY"), "Put GROQ_API_KEY in your .env"

client = Groq(api_key=os.getenv("GROQ_API_KEY"))
available = {m.id for m in client.models.list().data}

PREFERRED = [
    "llama-3.3-70b-versatile",
    "llama-3.1-8b-instant",
    "gemma2-9b-it",
]
model_name = next((m for m in PREFERRED if m in available), None)
if not model_name:
    raise RuntimeError(f"No preferred Groq models enabled. Have: {sorted(available)}")

llm = ChatGroq(model=model_name)
print("Using LLM:", model_name)


Using LLM: llama-3.3-70b-versatile


In [38]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.3-70b-versatile")  # relies on GROQ_API_KEY in env/.env
print(llm.invoke("Say hi in 3 words."))
llm.invoke("Tell me a joke about cats")

content='Hello to you' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 42, 'total_tokens': 46, 'completion_time': 0.025212753, 'prompt_time': 0.001865466, 'queue_time': 0.090877568, 'total_time': 0.027078219}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_155ab82e98', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--fc25e7bd-6427-4ac2-8f38-60d080248327-0' usage_metadata={'input_tokens': 42, 'output_tokens': 4, 'total_tokens': 46}


AIMessage(content='Why did the cat join a band? Because it wanted to be the purr-cussionist.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 41, 'total_tokens': 62, 'completion_time': 0.039433395, 'prompt_time': 0.002073381, 'queue_time': 0.091820913, 'total_time': 0.041506776}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_155ab82e98', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--82657350-56de-48a6-8bba-e7dba6a36d56-0', usage_metadata={'input_tokens': 41, 'output_tokens': 21, 'total_tokens': 62})

In [13]:

PDF_PATH = "data/Oppenheimer-2006-Applied_Cognitive_Psychology.pdf"  # <-- change to your PDF filename
CHUNK_SIZE = 1500
CHUNK_OVERLAP = 200




## Process PDF document

### Load PDF Models 

In [ ]:
# 3) Load PDF and split into chunks
loader = PyPDFLoader(PDF_PATH)
pages = loader.load()
pages

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,
                                            chunk_overlap=200,
                                            length_function=len,
                                            separators=["\n\n", "\n", " "])
chunks = text_splitter.split_documents(pages)
chunks

print(f"Loaded pages: {len(pages)} | Chunks: {len(chunks)}")


Ignoring wrong pointing object 18 0 (offset 0)


Loaded pages: 3 | Chunks: 9


In [40]:
from IPython.display import display

display(pages[:3])   # preview first 3 documents
display(chunks[:5]) # preview first 5 chunks
print(f"Loaded pages: {len(pages)} | Chunks: {len(chunks)}")

[Document(metadata={'producer': 'macOS Version 14.4.1 (Build 23E224) Quartz PDFContext, AppendMode 1.1', 'creator': 'Preview', 'creationdate': "D:20240909152042Z00'00'", 'author': 'Thu Vu', 'moddate': "D:20240910141854Z00'00'", 'title': 'Oppenheimer-2006-Applied_Cognitive_Psychology', 'source': 'data/Oppenheimer-2006-Applied_Cognitive_Psychology.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='APPLIED COGNITIVE PSYCHOLOGY\nAppl. Cognit. Psychol. 20: 139–156 (2006)\nPublished online 31 October 2005 in Wiley InterScience\n(www.interscience.wiley.com) DOI: 10.1002/acp.1178\nConsequences of Erudite Vernacular Utilized Irrespective\nof Necessity: Problems with Using Long Words Needlessly\nDANIEL M. OPPENHEIMER*\nPrinceton University, USA\nSUMMARY\nMost texts on writing style encourage authors to avoid overly-complex words. However, a majority\nof undergraduates admit to deliberately increasing the complexity of their vocabulary so as to give\nthe impression of intelligen

[Document(metadata={'producer': 'macOS Version 14.4.1 (Build 23E224) Quartz PDFContext, AppendMode 1.1', 'creator': 'Preview', 'creationdate': "D:20240909152042Z00'00'", 'author': 'Thu Vu', 'moddate': "D:20240910141854Z00'00'", 'title': 'Oppenheimer-2006-Applied_Cognitive_Psychology', 'source': 'data/Oppenheimer-2006-Applied_Cognitive_Psychology.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='APPLIED COGNITIVE PSYCHOLOGY\nAppl. Cognit. Psychol. 20: 139–156 (2006)\nPublished online 31 October 2005 in Wiley InterScience\n(www.interscience.wiley.com) DOI: 10.1002/acp.1178\nConsequences of Erudite Vernacular Utilized Irrespective\nof Necessity: Problems with Using Long Words Needlessly\nDANIEL M. OPPENHEIMER*\nPrinceton University, USA\nSUMMARY\nMost texts on writing style encourage authors to avoid overly-complex words. However, a majority\nof undergraduates admit to deliberately increasing the complexity of their vocabulary so as to give\nthe impression of intelligen

Loaded pages: 3 | Chunks: 9


## Create Embedings 



In [18]:
%pip install -U sentence-transformers
# If pip doesn't pull PyTorch automatically, install the CPU build explicitly:
# %pip install -U torch --index-url https://download.pytorch.org/whl/cpu


  Using cached sentence_transformers-5.1.1-py3-none-any.whl.metadata (16 kB)
  Using cached transformers-4.57.0-py3-none-any.whl.metadata (41 kB)
  Using cached torch-2.8.0-cp311-cp311-win_amd64.whl.metadata (30 kB)
  Using cached scikit_learn-1.7.2-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached regex-2025.9.18-cp311-cp311-win_amd64.whl.metadata (41 kB)
  Using cached safetensors-0.6.2-cp38-abi3-win_amd64.whl.metadata (4.1 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached markupsafe-3.0.3-cp311-cp311-win_amd64.whl.metadata (2.8 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
Using cached sentence_transformers-5.1.1-py3-none-any.whl (486 kB)
Using cached transformers-4.57.0-py3-none-any.whl (12.0 MB)
Using cached regex-2025.9.18-cp311-cp311-win_amd64.whl (276 kB)
Using cached safetensors-0.6.2-cp38-abi3-win_amd64.whl (320 kB)
Using cached torch-2.8.0-cp311-cp311-w

In [44]:
# pip install -U sentence-transformers langchain-community

from langchain_community.embeddings import HuggingFaceEmbeddings

def get_embedding_function(
    embed_model: str = "sentence-transformers/all-MiniLM-L6-v2",  # embedding model (not the LLM)
):
    return HuggingFaceEmbeddings(
        model_name=embed_model,
        encode_kwargs={"normalize_embeddings": True},   # cosine-normalized vectors
    )

embedding_function = get_embedding_function()
test_vector = embedding_function.embed_query("cat")
print("Dim:", len(test_vector), "| first 8:", test_vector[:8])


Dim: 384 | first 8: [0.03733031079173088, 0.051161788403987885, -0.0003060445887967944, 0.060209885239601135, -0.11749441176652908, -0.014230075292289257, 0.10577617585659027, 0.02678624354302883]


In [45]:
from langchain.evaluation import load_evaluator

evaluator = load_evaluator(evaluator="embedding_distance", 
                            embeddings=embedding_function)

evaluator.evaluate_strings(prediction="Amsterdam", reference="coffeeshop")

{'score': 0.7261297861891461}

In [46]:
evaluator.evaluate_strings(prediction="Paris", reference="coffeeshop")


{'score': 0.7031169469617871}

### Create Vector database 


In [25]:
import uuid

def create_vectorstore(chunks, embedding_function, vectorstore_path):

    # Create a list of unique ids for each document based on the content
    ids = [str(uuid.uuid5(uuid.NAMESPACE_DNS, doc.page_content)) for doc in chunks]
    
    # Ensure that only unique docs with unique ids are kept
    unique_ids = set()
    unique_chunks = []
    
    unique_chunks = [] 
    for chunk, id in zip(chunks, ids):     
        if id not in unique_ids:       
            unique_ids.add(id)
            unique_chunks.append(chunk) 

    # Create a new Chroma database from the documents
    vectorstore = Chroma.from_documents(documents=unique_chunks, 
                                        ids=list(unique_ids),
                                        embedding=embedding_function, 
                                        persist_directory = vectorstore_path)

    vectorstore.persist()
    
    return vectorstore

In [29]:
# Create vectorstore
vectorstore = create_vectorstore(chunks=chunks, 
                                 embedding_function=embedding_function, 
                                 vectorstore_path="vectorstore_chroma")

## 2. Query for relevant data


In [30]:
# Load vectorstore
vectorstore = Chroma(persist_directory="vectorstore_chroma", embedding_function=embedding_function)

In [31]:
# Create retriever and get relevant chunks
retriever = vectorstore.as_retriever(search_type="similarity")
relevant_chunks = retriever.invoke("What is the title of the paper?")
relevant_chunks

[Document(metadata={'page_label': '2', 'page': 1, 'source': 'data/Oppenheimer-2006-Applied_Cognitive_Psychology.pdf', 'title': 'Oppenheimer-2006-Applied_Cognitive_Psychology', 'moddate': "D:20240910141854Z00'00'", 'total_pages': 3, 'author': 'Thu Vu', 'creator': 'Preview', 'creationdate': "D:20240909152042Z00'00'", 'producer': 'macOS Version 14.4.1 (Build 23E224) Quartz PDFContext, AppendMode 1.1'}, page_content='was unnecessary and thus surprising readers with the relative disﬂuency of the text.\nBoth the experts and prevailing wisdom present plausible views, but which (if either) is\ncorrect? The present paper provides an empirical investigation of the strategy of complex-\nity, and ﬁnds such a strategy to be unsuccessful. Five studies demonstrate that the loss of\nﬂuency due to needless complexity in a text negatively impacts raters’ assessments of the\ntext’s authors.\nEXPERIMENT 1\nExperiment 1 aimed to answer several simple questions. First, does increasing the\ncomplexity of tex

In [33]:
# Prompt template
PROMPT_TEMPLATE = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer
the question. If you don't know the answer, say that you
don't know. DON'T MAKE UP ANYTHING.


{context}

---

Answer the question based on the above context: {question}
"""

## 3.Generate responses

In [47]:
# Concatenate context text
from langchain_core.prompts import ChatPromptTemplate


context_text = "\n\n---\n\n".join([doc.page_content for doc in relevant_chunks])

# Create prompt
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, 
                                question="What is the name of the author?")
print(prompt)

Human: 
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer
the question. If you don't know the answer, say that you
don't know. DON'T MAKE UP ANYTHING.


was unnecessary and thus surprising readers with the relative disﬂuency of the text.
Both the experts and prevailing wisdom present plausible views, but which (if either) is
correct? The present paper provides an empirical investigation of the strategy of complex-
ity, and ﬁnds such a strategy to be unsuccessful. Five studies demonstrate that the loss of
ﬂuency due to needless complexity in a text negatively impacts raters’ assessments of the
text’s authors.
EXPERIMENT 1
Experiment 1 aimed to answer several simple questions. First, does increasing the
complexity of text succeed in making the author appear more intelligent? Second, to
what extent does the success of this strategy depend on the quality of the original, simpler
writing? Finally, if the strategy is unsuccessful, is t

In [48]:
llm.invoke(prompt)

AIMessage(content='The name of the author is D. M. Oppenheimer.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 1320, 'total_tokens': 1335, 'completion_time': 0.019747276, 'prompt_time': 0.109632318, 'queue_time': 0.0895196, 'total_time': 0.129379594}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_9e1e8f8435', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--2f9f6fea-ef19-40ee-9c3c-e74667646360-0', usage_metadata={'input_tokens': 1320, 'output_tokens': 15, 'total_tokens': 1335})

## Generate structured responses


In [55]:
from pydantic import BaseModel, Field

class AnswerWithSources(BaseModel):
    """An answer to the question, with sources and reasoning."""
    answer: str = Field(description="Answer to question")
    sources: str = Field(description="Full direct text chunk from the context used to answer the question")
    reasoning: str = Field(description="Explain the reasoning of the answer based on the sources")
    
class ExtractedInfo(BaseModel):
    """Extracted information about the research article"""
    paper_title: AnswerWithSources
    paper_summary: AnswerWithSources
    publication_year: AnswerWithSources
    paper_authors: AnswerWithSources

In [57]:
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt_template
            | llm.with_structured_output(ExtractedInfo, strict=True)
        )

rag_chain.invoke("Give me the title, summary, publication date, authors of the research paper.")

ExtractedInfo(paper_title=AnswerWithSources(answer='The Effect of Complexity on Judgments of Authors', sources='The study did not look at how readers of the texts containing the long words perceived the authors’ intelligence.', reasoning='The title of the paper is not explicitly stated, but based on the content, it can be inferred to be related to the effect of complexity on judgments of authors.'), paper_summary=AnswerWithSources(answer='The paper investigates the strategy of using complex language to appear more intelligent and finds it to be unsuccessful.', sources='Five studies demonstrate that the loss of fluency due to needless complexity in a text negatively impacts raters’ assessments of the text’s authors.', reasoning='The paper presents an empirical investigation of the strategy of complexity and finds that it is unsuccessful in making the author appear more intelligent.'), publication_year=AnswerWithSources(answer='2006', sources='Copyright # 2005 John Wiley & Sons, Ltd. App

## Transform response into a dataframe

In [61]:
structured_response = rag_chain.invoke("Give me the title, summary, publication date, authors of the research paper.")
df = pd.DataFrame([structured_response.dict()])

# Transforming into a table with two rows: 'answer' and 'source'
answer_row = []
source_row = []
reasoning_row = []

for col in df.columns:
    answer_row.append(df[col][0]['answer'])
    source_row.append(df[col][0]['sources'])
    reasoning_row.append(df[col][0]['reasoning'])

# Create new dataframe with two rows: 'answer' and 'source'
structured_response_df = pd.DataFrame([answer_row, source_row, reasoning_row], columns=df.columns, index=['answer', 'source', 'reasoning'])
structured_response_df

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_27072\1998849655.py:2: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  df = pd.DataFrame([structured_response.dict()])


,paper_title,paper_summary,publication_year,paper_authors
answer,The Effects of Complexity on Writing,The paper investigates the strategy of complex...,2006,D. M. Oppenheimer
source,The study did not look at how readers of the t...,Five studies demonstrate that the loss of flue...,"Copyright # 2005 John Wiley & Sons, Ltd. Appl....",140 D. M. Oppenheimer
reasoning,"The title is not explicitly stated, but based ...",The paper presents several studies that demons...,The publication year is mentioned in the text ...,The author is mentioned in the text as D. M. O...
